In [2]:
import requests
from bs4 import BeautifulSoup

In [71]:
import csv
import json

In [2]:
URL = 'https://www.sports-reference.com/cfb/schools/southern-california/'

In [4]:
r = requests.get(URL)
soup = BeautifulSoup(r.text, "html5lib")


In [14]:
coaches = soup.find_all(attrs={"data-stat": "coaches"})

In [20]:
c = coaches[5]

In [23]:
c

<td class="left " csk="Helton,Clay.2013 Kiffin,Lane.2013 Orgeron,Ed.2013" data-stat="coaches"><a href="/cfb/coaches/clay-helton-1.html">Clay Helton</a> (1-0), <a href="/cfb/coaches/lane-kiffin-1.html">Lane Kiffin</a> (3-2), <a href="/cfb/coaches/ed-orgeron-1.html">Ed Orgeron</a> (6-2)</td>

In [21]:
list(c.children)

[<a href="/cfb/coaches/clay-helton-1.html">Clay Helton</a>,
 u' (1-0), ',
 <a href="/cfb/coaches/lane-kiffin-1.html">Lane Kiffin</a>,
 u' (3-2), ',
 <a href="/cfb/coaches/ed-orgeron-1.html">Ed Orgeron</a>,
 u' (6-2)']

In [22]:
len(list(c.children))

6

In [24]:
c.text

u'Clay Helton (1-0), Lane Kiffin (3-2), Ed Orgeron (6-2)'

In [33]:
tbl = soup.find("table", "stats_table").tbody.find_all('tr')

In [48]:
teams = []
with open('sportsref-teams.psv') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='|')
    for row in reader:
        teams.append(row)

In [71]:
coaches = []
with open('sportsref-coaches.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if int(row['year']) >= 2000 and int(row['year']) < 2017:
            coaches.append(row)


In [69]:
def parse_coach(string):
    nr = string[:-1].split(" (")
    name = nr[0]
    record = nr[1]
    wl = record.split("-")
    wins = wl[0]
    losses = wl[1]
    
    return name, int(wins), int(losses)

In [13]:
coaches = []
with open('coaches-ordered-corrected.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        cleaned = {}
        cleaned['team'] = row['team']
        cleaned['season'] = row['year']
        cleaned['coach'] = row['name']
        cleaned['games'] = int(row['total_games'])
        if row['correct_order'] == '':
            cleaned['order'] = int(row['order'])
        else:
            cleaned['order'] = int(row['correct_order'])
        
        coaches.append(cleaned)

In [11]:
bool("")

False

In [14]:
coaches_sorted = sorted(coaches, key=lambda x: x['order'])

In [17]:
coaches_sorted[-10:]

[{'coach': 'DeWayne Walker',
  'games': 1,
  'order': 2,
  'season': '2007',
  'team': 'ucla'},
 {'coach': 'Ed Kezirian',
  'games': 1,
  'order': 2,
  'season': '2002',
  'team': 'ucla'},
 {'coach': 'Clay Helton',
  'games': 9,
  'order': 2,
  'season': '2015',
  'team': 'southern-california'},
 {'coach': 'Ed Orgeron',
  'games': 8,
  'order': 2,
  'season': '2013',
  'team': 'southern-california'},
 {'coach': 'Kyle Whittingham',
  'games': 1,
  'order': 2,
  'season': '2004',
  'team': 'utah'},
 {'coach': 'Marques Tuiasosopo',
  'games': 1,
  'order': 2,
  'season': '2013',
  'team': 'washington'},
 {'coach': 'Bill Stewart',
  'games': 1,
  'order': 2,
  'season': '2007',
  'team': 'west-virginia'},
 {'coach': 'Barry Alvarez',
  'games': 1,
  'order': 2,
  'season': '2014',
  'team': 'wisconsin'},
 {'coach': 'Barry Alvarez',
  'games': 1,
  'order': 2,
  'season': '2012',
  'team': 'wisconsin'},
 {'coach': 'Clay Helton',
  'games': 1,
  'order': 3,
  'season': '2013',
  'team': 'sout

In [25]:
team_look = {}

In [26]:
for c in coaches_sorted:
    t = c['team']
    s = c['season']
    g = c['games']
    ch = c['coach']
    if not team_look.get(t):
        team_look[t] = {}
    if not team_look[t].get(s):
        team_look[t][s] = []
    for i in range(0, g):
        team_look[t][s].append(ch)
    
    

In [48]:
class Tenure(object):
    def __init__(self, info):
        self.coach = info['coach']
        self.results = []
        self.currszn = info['season']
        self.seasons = 1
        self.team = info['team']
        self.min_year = info['season']
        self.max_year = info['season']
        self.left_censor = (info['game_num'] == '1') and (info['season'] == '2000')
        self.right_censor = False
    
    def win(self):
        self.results.append(True)
    
    def loss(self):
        self.results.append(False)
    
    def add_szn(self):
        self.seasons += 1
    
    def slugify(self):
        return "{}|{}|{}|{}".format(
            self.coach.lower().split()[-1],
            self.min_year[2:4],
            self.max_year[2:4],
            self.team
        )

In [56]:
games = []
with open('sportsref-gamelogs.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        row['coach'] = team_look[row['team']][row['season']][int(row['game_num'])-1]
        games.append(row)

In [57]:
games = sorted(games, key=lambda x: (x['team'], x['date']))

In [68]:
tenures = []
T = Tenure(games[0])

for gm in games[1:]:
    newcoach = (gm['coach'] != T.coach)
    newteam = (gm['team'] != T.team)
    
    if newcoach or newteam:
        tenures.append(T)
        T = Tenure(gm)
    
    if gm['season'] != T.currszn:
        T.add_szn()
        T.currszn = gm['season']
        T.max_year = gm['season']
    
    if gm['result'] == 'W':
        T.win()
    else:
        T.loss()
        
    if (int(gm['game_num']) == len(team_look[gm['team']][gm['season']])) and (gm['season'] == '2016'):
        T.right_censor = True
        T.max_year = gm['season']
        tenures.append(T)

In [69]:
[{
    'coach': t.coach,
    'games': len(t.results),
    'wins': sum(t.results),
    'losses': len(t.results) - sum(t.results),
    'currszn': t.currszn,
    'seasons': t.seasons,
    'team': t.team,
    'min_year': t.min_year,
    'max_year': t.max_year,
    'left_censor': t.left_censor,
    'right_censor': t.right_censor,
    'slug': t.slugify()
} for t in tenures]

[{'coach': 'Mike DuBose',
  'currszn': '2000',
  'games': 10,
  'left_censor': True,
  'losses': 7,
  'max_year': '',
  'min_year': '00',
  'right_censor': False,
  'seasons': 1,
  'slug': 'dubose|00||alabama',
  'team': 'alabama',
  'wins': 3},
 {'coach': 'Dennis Franchione',
  'currszn': '2002',
  'games': 24,
  'left_censor': False,
  'losses': 8,
  'max_year': '02',
  'min_year': '01',
  'right_censor': False,
  'seasons': 2,
  'slug': 'franchione|01|02|alabama',
  'team': 'alabama',
  'wins': 16},
 {'coach': 'Mike Shula',
  'currszn': '2006',
  'games': 49,
  'left_censor': False,
  'losses': 23,
  'max_year': '06',
  'min_year': '03',
  'right_censor': False,
  'seasons': 4,
  'slug': 'shula|03|06|alabama',
  'team': 'alabama',
  'wins': 26},
 {'coach': 'Joe Kines',
  'currszn': '2006',
  'games': 1,
  'left_censor': False,
  'losses': 1,
  'max_year': '',
  'min_year': '06',
  'right_censor': False,
  'seasons': 1,
  'slug': 'kines|06||alabama',
  'team': 'alabama',
  'wins': 0}